In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
import pandas as pd
import pathlib
import numpy as np

data_dir = pathlib.Path().resolve().parent/"data"
model_dir = pathlib.Path().resolve().parent/"models"



In [28]:
data = pd.read_csv(data_dir/"olympic\preprocessed_data.csv")

data.head(1)

,Sun,Moon,Mercury,Venus,Mars,Jupiter,Saturn,Uranus,Neptune,Pluto,Lilith,Ascending node,Descending node,won
0,"[223.17501312036723, 64.51883847705251, 223.17...","[54.43503142165456, 279.45355900765236, 54.435...","[245.25266971211485, 39.775896652933575, 245.2...","[196.88817247928412, 50.45674611110462, 196.88...","[286.6504682365322, 62.537560942791266, 286.65...","[174.67587251009869, 116.76900433551279, 174.6...","[297.1558890408487, 290.76620531635166, 297.15...","[130.5153210355428, 118.2395267189684, 130.515...","[26.711937028289018, 25.239584612953863, 26.71...","[50.91713149181603, 49.6964148199364, 50.91713...","[170.30211351617362, 111.14680700521824, 170.3...","[44.94315930125862, 72.59262932821451, 44.9431...","[44.99476290951289, 73.00638583180817, 44.9947...",False


In [30]:
def create_graph_data(row):
    # Извлечение признаков участников команды
    features = []
    for col in data.columns[:-1]:
        features.extend(eval(row[col]))

    # Преобразование признаков в тензор
    x = torch.tensor(features, dtype=torch.float).view(-1, 13)

    # Создание графа
    edge_index = torch.tensor([[i, j] for i in range(len(x)) for j in range(len(x)) if i != j], dtype=torch.long).t().contiguous()

    # Метка успешности команды
    y = torch.tensor([row['won']], dtype=torch.float)

    return Data(x=x, edge_index=edge_index, y=y)

graphs = [create_graph_data(row) for _, row in data.iterrows()]

In [32]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

train_size = int(0.8 * len(graphs))
train_graphs = graphs[:train_size]
test_graphs = graphs[train_size:]

# Создание DataLoader для обучающей и тестовой выборок
train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

In [33]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GroupSuccessPredictor(nn.Module):
    def __init__(self, num_features, hidden_dim, output_dim):
        super(GroupSuccessPredictor, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return torch.sigmoid(x)

model = GroupSuccessPredictor(num_features=13, hidden_dim=32, output_dim=1)

In [34]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [36]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        data.y = data.y.view(-1, 1)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

Epoch 1, Loss: 0.6593500228627963
Epoch 2, Loss: 0.602130371563179
Epoch 3, Loss: 0.5718914251432266
Epoch 4, Loss: 0.5525221269872728
Epoch 5, Loss: 0.5392509757267604
Epoch 6, Loss: 0.5311002576685733
Epoch 7, Loss: 0.5244285217046021
Epoch 8, Loss: 0.5128696447545137
Epoch 9, Loss: 0.5080119922652596
Epoch 10, Loss: 0.502262230819378
Epoch 11, Loss: 0.49727276318594243
Epoch 12, Loss: 0.49129217487193255
Epoch 13, Loss: 0.4885772257829206
Epoch 14, Loss: 0.4836297652348403
Epoch 15, Loss: 0.48238413276837255
Epoch 16, Loss: 0.4769890545066129
Epoch 17, Loss: 0.47474910737059806
Epoch 18, Loss: 0.47154096990943234
Epoch 19, Loss: 0.4683005144309926
Epoch 20, Loss: 0.4667375670192714
Epoch 21, Loss: 0.4646532555965528
Epoch 22, Loss: 0.4643363251764361
Epoch 23, Loss: 0.46246729274129456
Epoch 24, Loss: 0.46116169601320384
Epoch 25, Loss: 0.4601592824928465
Epoch 26, Loss: 0.4561563365975432
Epoch 27, Loss: 0.45510516204425333
Epoch 28, Loss: 0.45574856556969423
Epoch 29, Loss: 0.4542

In [37]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        output = model(data)
        data.y = data.y.view(-1, 1)
        predicted = (output > 0.5).float()
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 62.95%


In [ ]:
torch.save(model.state_dict(), model_dir/"model.pth")